# Ray Tracing and Channel Data Exploration

Evaluation of wireless systems requires simulation in realistic environments.  In modern research, the trend has been use ray tracing where propagation is simulated based on a 3D model of the environment.  A valuable feature of Nvidia's sionna package is that has integrated a [ray tracing module](https://nvlabs.github.io/sionna/api/rt.html) module that can be used with the communications library to evaluate complex systems in actual environments.  Unfortunately, as you will see in the simulations below, the performance of Sionna is very slow compared to a commercial ray tracer like Remcom's.  Also, on most machines, it can only handle a very limited number of data points.  Still it is an excellent learning tool, easy-to-use, and completely free.

Nvidia's sionna documentation already has many examples that you can use a template for your work.  In this lab, we will look at a few additional features.  Specifically, in going through this lab, you will learn to:

* Read and display scene
* Create a grid of locations and identify which points are indoor and outdoor
* Characterize the link state and plot the link state probability vs distance
* Compute the total path loss and plot the path loss vs. distance

*Submission*: Complete all the sections marked `TODO`, and run the cells to make sure your scipt is working. When you are satisfied with the results,  publish your code to generate an html file. Print the html file to PDF and submit the PDF.


## Importing the Sionna Package
We first follow the demo and import the sionna package.  If you are running this notebook on your own machine, you should install sionna.  The code below will install sionna on Google colab.

In [ ]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna as sn
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    !pip install sionna
    #os.system("pip install sionna")
    import sionna as sn

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

While sionna has a number of features for coverage maps, I needed to add some custom capabilities.  I added these to a `sionnautils` package in the class github. We import this package as well.

In [ ]:
# Import Sionna utils from the wireless class
try:
    import sionnautils
except ImportError as e:
    # Install Sionna if package is not already installed
    !pip install git+https://github.com/sdrangan/wirelesscomm.git
    import sionnautils


In [ ]:

# Colab does currently not support the latest version of ipython.
# Thus, the preview does not work in Colab. However, whenever possible we
# strongly recommend to use the scene preview mode.
try: # detect if the notebook runs in Colab
    import google.colab
    no_preview = True # deactivate preview
except:
    if os.getenv("SIONNA_NO_PREVIEW"):
        no_preview = True
    else:
        no_preview = False

resolution = [480,320] # increase for higher quality of renderings

# Define magic cell command to skip a cell if needed
from IPython.core.magic import register_cell_magic
from IPython import get_ipython

@register_cell_magic
def skip_if(line, cell):
    if eval(line):
        return
    get_ipython().run_cell(cell)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Load the Scene
We will follow the [Nvidia sionna demo](https://nvlabs.github.io/sionna/examples/Sionna_Ray_Tracing_Introduction.html) and load a scene.  We will use one of Nvidia's sionna's in-built scenes of the center of Munich.  However, Nvidia also has a nice [video](https://www.youtube.com/watch?v=7xHLDxUaQ7c) on building your own scenes from OpenStreetMap and Blender.

In [ ]:
# Load integrated scene
scene = sn.rt.load_scene(sn.rt.scene.munich)

We can view the scene.  If you are using Google colab, colab does not support interactive viewers.  In this case, the following cell can be executed to provide a static view of the scene.  If you are on a different system, you may be able to use the scene preview where you can use the interact with the map.

In [ ]:
# Render scene
if no_preview:
    scene.render(camera="scene-cam-0", num_samples=512);

In [ ]:
%%skip_if no_preview
# Open 3D preview (only works in Jupyter notebook)
scene.preview()

## Creating a Coverage Grid

For this lab, we will make measurements on a grid of points.  Sionna has a [coverage map](https://nvlabs.github.io/sionna/api/rt.html#coverage-maps) class, but I found it to be not very flexible.  So, I wrote a simple class that help create grid points and classify them.  You can create the grid points with the following commands.  In this lab, we use a coarse grid at `10m` spacing.  But, in reality, you would want to make the spacing a little smaller.  I used the `10m` grid only to make this lab run relatively fast.

The `cm.set_grid()` method creates a set of grid points of points to run analysis.  The `cm.compute_grid_attributes()` performs some simple geometric classification on each grid point.  This method may take a few minutes for a dense grid, but for this experiment it should run in a few seconds since the grid is not very coarse.  Also, to simplify the analysis later, we have the set the bounding box of the grid to a relatively small region of the scene.  If you set, `bbox = None`, it will use the full scene area.

In [ ]:
from sionnautils.miutils import CoverageMapPlanner

grid_size = 5

# Set a bounding box in the center bbox = [xmin, xmax, ymin, ymax]
bbox = [-200,200,-200,200]

cm = CoverageMapPlanner(scene._scene, grid_size=grid_size, bbox=bbox)
cm.set_grid()
cm.compute_grid_attributes()

Running the above command will create a few useful arrays:
* `cm.nx, cm.ny` : Number of grid points in the `x` and `y` directions
* `cm.x` :  a `nx`-dimension vector of the `x`-values of the grid points
* `cm.y` :  a `ny`-dimension vector of the `y`-values of the grid points
* `cm.xgrid`, `cm.ygrid`: `(ny,nx)` grid of points with the `x` and `y` values of the grid points
* `cm.zmin_grid`, `cm.zmax_grid`:  `(ny,nx)` grids of values indicating the maximum and minimum value of `z` values of objects in the scene at the grid points.  `cm.zmin_grid[i,j]` is generally the elevation of the ground at the location `(i,j)` and `cm.zmax_grid[i,j]` is the elevation of the rooftop at `(i,j)`.
* `cm.bldg_grid`:  A `(ny,nx)` grid of values where `cm.bldg_grid[i,j]==1` indicates if the point is estimated to be an outdoor location.  A point `(i,j)` is assumed to be outdoor if `cm.zmin[i,j] ~= cm.zmax[i,j]`.
* `cm.in_region`:  A `(ny,nx)` grid of values where `cm.in_region[i,j]==1` ndicates if the point is estimated to in the region of the buildings, meaning that there is at least one non-outdoor point to the left and right of the point.

You can display this map as follows:

* Create an array `point_type` where `point_type[i,j] == 0` if the outside the region (i.e., `cm.in_region[i,j]==False`); `point_type[i,j] == 1` if inside the region and not a building; and `point_type[i,j] == 2` if the point is a building.
* Flip the `point_type` array with `point_type = np.flipud(point_type)`.  We do this since the `imshow` command expects the larger values of `y` to be at the top.
* Plot `point_type` with `plt.imshow(point_type, cmap=cmap, extent=...)`.  You will want to set the `extent=[xmin, xman, ymin, ymax]` which you can get from the vectors `cm.x` and `cm.y`.  The `cmap` parameter sets the color map.
* Label the `x` and `y` axes.
* If you want to be fancy, add a legend, but you don't have to do that.

In [ ]:
# Define a custom color map with RGB values
from matplotlib.colors import ListedColormap
colors = ['lightgray', 'lightblue', 'brown']
cmap = ListedColormap(colors)


# TODO:  Create a grid of point types
#   point_type = ...

# TODO:  Plot the grid
#   extent = ...
#   plt.imshow(...)



## Creating a Set of Test Transmitters and Receivers
In simulating propagation in cellular systems, we often place a number of transmitters in typical locations for base stations, and receivers in locations where users may have a handset.  In this case, we will place the transmitters at random locations on top of rooftops.  To do this:

* Find `ntx` random indices `(i,j)` which correspond to a building.  That is, `cm.bldg_grid[i,j]==True`
* For each location, find the `x` and `y` locations.  Note that a point `(i,j)` is at the `(x,y)=(cm.x[j], cm.y[i])`.  Note the reversal in `(i,j)`.
* For each location, set the `z` value to `cm.zmax_grid[i,j] + height_above_roof`.  This will place the transmitter above the roof similar to a typical base station.  Here, we have placed a bit higher than usual, but we are not selecting the rooftop location particularly well.
* Store the final set of locations in an array `tx_pos` which should be `(ntx,3)`.

In [ ]:
ntx = 3
height_above_roof = 10

# TODO:  Find ntx random indices
#   tx_ind = ...


#  TODO:  Set the tx positions
#   tx_pos = ...

To validate the selection, plot the map `point_type` again.  On top of the plot, plot the markers (say small blue circles) where the transmitters are.  You should see that they are on the building locations

In [ ]:
# TODO:  PLot the area with the TX
#   plt.imshow(point_type, ...)
#   plt.scatter(...)

Next, we add the receivers:

* Find `nrx` random indices `(i,j)` which correspond to a point in the region, but not a building.  That is, `cm.bldg_grid[i,j]==False` and `cm.in_region[i,j] == True`.
* For each location, find the `x` and `y` locations. 
* For each location, set the `z` value to `cm.zmin_grid[i,j] + height_above_ground`.  This will place the receiver slightly above ground, about the distance is cell phone is carried.
* Store the final set of locations in an array `rx_pos` which should be `(nrx,3)`.
* Create a plot with the TX and RX locations

In [1]:
nrx = 100

# TODO:  Find the RX locations
#   rx_pos = ...


# TODO:  Plot the map with the TX and RX locations

## Running the Ray Tracing
Now that we have a 3D environment and TX and RX locations, we can run the ray tracing.

First, we configure the antennas and add the receivers. We will add the transmitters later.  For the antennas use vertically polarized isotropic antennas.

In [ ]:
# Frequency in Hz
fc = 2.5e9

scene.frequency = fc # in Hz; implicitly updates RadioMaterials
scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element

# TODO:  Configure antenna array for all transmitters.  Use an iso pattern with V-pol
#   scene.tx_array = sn.rt.PlanarArray(...)
scene.tx_array = sn.rt.PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

# TODO:  Configure antenna array for all receivers similarly
#    scene.rx_array = sn.rt.PlanarArray(...)
scene.rx_array = sn.rt.PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

# Remove all previous receivers
for rx_name in scene.receivers:
    scene.remove(rx_name)
    
# TODO:  Add receivers at the locations rx_pos


Next we compute the paths.  We run the ray tracing in a number of *runs*, with one TX in each run.  At least on my computer, sionna seems to struggle with even a small number of links to simulate.  So, we need to break the simulation into smaller runs.  I am not sure if there is something wrong in the configuration.  

*Warning*:  In addition to the limited capacity of sionna, sometimes if you run the same cell many times, Nvidia's sionna will throw a `ResourceExhaustedError`.  My feeling is that this is a memory leak as the code does not seem to clear the memory.    If you run into this, restart the kernel from the beginning.

In [ ]:
# Parameters
diffraction = False
scattering = False
num_samples = 1e6
max_depth = 5
scene.synthetic_array = True

path_dict_list  = []

# Loop over runs
for i in range(ntx):
    print('Tx = %d' % i)
    
    # Remove all previous transmitters
    for tx_name in scene.transmitters:
        scene.remove(tx_name)

    # TODO:  Add a transmitter at position tx_pos[i]
    #  tx = ...
    #  scene.add(tx)
    tx = sn.rt.Transmitter(name=f"tx-{i}",
                  position=tx_pos[i])
    scene.add(tx)

    # Run ray tracing with this transitter
    paths = scene.compute_paths(max_depth=max_depth,
                            diffraction=diffraction,
                            scattering=scattering,
                            num_samples=num_samples)

    paths.normalize_delays = False

    # Extract a dictionary of the path data
    path_dict = paths.to_dict()

    # Save the path dictionary
    path_dict_list.append(path_dict)

Since the ray tracing takes a long time, it is useful to save the results in case we need them later.

In [ ]:
import pickle

# Save the variable to a file
with open('rt_data.pkl', 'wb') as file:
    pickle.dump([rx_pos, tx_pos, path_dict_list], file)
    


We can load it as follows

In [ ]:
# Load the variable from the file
with open('rt_data.pkl', 'rb') as file:
    rx_pos, tx_pos, path_dict_list = pickle.load(file)

## Classifying Link State

A *link* is a channel from a single TX to asingle RX.  Typically, links can be in one of three states:
* *Line-of-sight (LOS)*:  if there is a direct path from the TX to RX without any interactions such as reflections, diffractions, or transmission.
* *Non-line-of-sight (NLOS)* meaning there is at least one path from the TX to RX, but no LOS path.
* *Outage* if there are no paths of significant power

As a first step in analyzing the paths, we show how to classify each link. To do this, we will use the data in the `path_dict_list` which is a list of dictionaries with one dictionary per run.  Complete the following code to print the elements of the dictionary for the first run.  Get the first dictionary in the list and print its keys.

In [ ]:
path_dict = path_dict_list[0]  # The first path dictionary

# TODO:  print the keys in path_dict.keys()

You will that the dictionary `path_dict` has many entries for different characteristics of the paths.  For the link state, we will use the `path_dict[`tau`]` array which is of the shape:
~~~
    nbatch, nrx0, ntx0, npath_max0 = path_dict[`tau`].shape
~~~
where 
* `nbatch` is the number of samples per batch.  For our simulation, it is one
* `nrx0` is the number of receivers.  In our case, `nrx0=nrx` since each run had all the receivers
* `ntx0` is the number of transmitters in the run.  In our case, it is one
* `npath_max0` is the maximum number of paths.
By covention, if `tau=path_dict['tau']`, a component in the array `tau[0,irx.itx,ipath] > 0` if the path is valid.  A negative value indicates no path.  Also, the first path  `tau[0,irx.itx,0]` always corresponds to the LOS.

We can also use:
* `path_dict['sources']` is an `(ntx0,3)` array of the locations of all the transmitters in the run
* `path_dict['targets']` is an `(nrx0,3)` array of the locations of all the receivers in the run

To classify the links:
* Convert `path_dict['tau']`, `path_dict['targets']` and `path_dict['sources']` to numpy arrays `tau`, `rx_pos0`, `tx_pos0`.
* Using `rx_pos0` and `tx_pos0` create a matrix `dist0` of shape `(nrx0,ntx0)` of the distances between the TX and RXs.
* A link is LOS if the `tau[0,irx,itx,0] > 0`.  Similarly, a link is in outage if `tau[0,irx,itx,ipath] < 0` for all paths `ipath`.  Use this to create an array `link_state0` where `link_state0[irx,itx] = outage_state, los_state` or `nlos_state` depending on the link state.
* Create a scatter plot of `link_state0` vs. `dist0`. 

You should see that the LOS links are more common at closer distances and the outage links are more common at larger distances.

In [ ]:
# Link states
outage_state = 0
nlos_state = 1
los_state = 2
nlink_states = 3


# TODO:  Convert `path_dict['tau']`, `path_dict['sources']` and `path_dict['targets']` to numpy arrays
#   tau = ...
#   tx_pos0 = ...
#   rx_pos0 = ...

# TODO:  Create an array of distances 
#  dist0 = ...


# TODO:  Create an array of link state values
#   link_state0 = ...


# TODO:  Create a scatter plot of the values in link_state0 vs. dist0


Now, repeat the above code for all the runs in `path_dict_list`.  Aggregate the results into vectors `dist` and `link_state`.  The total number of entries in each vector should `nrx*ntx`.  

In [ ]:
dist = np.zeros((0,))
link_state = np.zeros((0,))

for path_dict in path_dict_list:
    # TODO: Get tau, tx_pos0 and rx_pos0
     
    # TODO:  Create an array of distances 
    #  dist0 = ...
    
    # TODO:  Create an array of link state values
    #   link_state0 = ...

    # TODO:  Flatten dist0 and link_state0 and append to 
    # dist and link_state with np.concatenate()
 

Now display the link state probability as a function of distance via a bar graph.  As an example, your graph could look as follows:

<img src="https://raw.githubusercontent.com/sdrangan/wirelesscomm/master/unit02_propagation/link_state_hist.png" width="400">

In [ ]:
# TODO:  Create a probability plot as above


## Examining Path Loss vs. Distance

In this final section, we will measure the total received power on all paths.  The path data is the array `path_dict['a']`.  If we set `a = path_dict['a']` it will have the shape:
~~~
   (nbatch, nrx0, nant_rx, ntx0, nant_tx, npath_max, ntime)
~~~
In our simulation, all but `nrx0`, `ntx0`, and `npath_max` are 1.  Each value is `a[0,irx,0,itx,0,j,0]` is the complex gain on path `j` between TX `itx` and RX `irx`.  By summing the magnitude squared of the gains along the paths, we can compute the total path gain, or equivalently, the path loss.  Complete the code below to compute the path loss across all the links.  Note that the values in the array `a` are independent of the TX power.


In [ ]:
path_loss = np.zeros((0,))

for path_dict in path_dict_list:
    # TODO: a and convert to numpy
    #  a = ...
 
    # Compute the total RX power
    #  rx_pow0 = ... 
    
    # TODO:  Compute the path loss in dBm.  When rx_pow0[irx,itx] == 0,
    # set the path loss to some high value.
    #   path_loss0

    # TODO:  Flatten path_loss0 and concatenate with path_loss


On a single plot, plot:
* A scatter plot of `path_loss` vs. `distance` for the LOS points.
* A scatter plot of `path_loss` vs. `distance` for the NLOS points.
* Path loss vs. distance for Friis law

Use `semilogx` to keep the distance in logarithmic scale  Put a legend on your plot.

In [2]:
# TODO:  Create the plot as described above
